In [1]:
import numpy as np
import imageio
import astra

vol_dim = 256

try:
    # volume = np.load(f"shepp_logan_{vol_dim}.npy")
    volume = np.load(f"SIRT/ref_au_ag.npy")
except:
    # from phantominator import shepp_logan
    # volume = shepp_logan((vol_dim, vol_dim, vol_dim))
    # np.save(f"shepp_logan_{vol_dim}.npy", volume)
    pass

range_ = 180
rest = -range_/2 
step = 1
angles = np.arange(rest, (range_+rest if rest < 0 else range_-rest)+step, step) 
num_projections = len(angles)

proj_geom = astra.create_proj_geom('parallel3d', 1.0, 1.0, vol_dim, vol_dim, np.radians(angles))
vol_geom  = astra.create_vol_geom(vol_dim, vol_dim, vol_dim)

projector_id = astra.create_projector('cuda3d', proj_geom, vol_geom)
sinogram_id, sinogram = astra.create_sino3d_gpu(volume, proj_geom, vol_geom)

astra.data3d.delete(sinogram_id)
astra.projector.delete(projector_id)

sinogram = np.swapaxes(sinogram, 0, 1) # [H, N, W] -> [N, H, W]
imageio.mimwrite(f'data/benchmark/au_ag_sirt_{range_}.tiff', sinogram)
# imageio.mimwrite(f'data/shepp_projections/shepp_projection_{vol_dim}_{range_}.tiff', sinogram)
sinogram_8bit = (255 * (sinogram - np.min(sinogram)) / (np.max(sinogram) - np.min(sinogram))).astype(np.uint8)
imageio.mimwrite(f'data/benchmark/au_ag_sirt_{range_}_8bit.tiff', sinogram_8bit)
# imageio.mimwrite(f'data/shepp_projections/shepp_projections_{vol_dim}_{range_}_8bit.tiff', sinogram_8bit)

with open(f'data/benchmark/au_ag_sirt_{range_}.txt', 'w') as angle_file:
    for angle in angles:
        angle_file.write(f"{angle}\n")

In [33]:

# # Define the parameters
# vol_dim = 256  # Example volume dimensions
# angles = np.arange(0, 5, 0.5)  # Angle range from 0 to 5 degrees with 0.5 degree steps
# num_projections = len(angles)
# det_row_count = vol_dim  
# det_col_count = vol_dim 

# det_spacing_x = 1.0
# det_spacing_y = 1.0

# source_origin_dist = 1000  # Distance from source to origin
# origin_detector_dist = 500  # Distance from origin to detector

# # Corrected call to create_proj_geom
# proj_geom = astra.create_proj_geom('cone', det_spacing_x, det_spacing_y, det_row_count, det_col_count, np.radians(angles), 
#                                    source_origin_dist, origin_detector_dist)

# # Create volume geometry
# vol_geom = astra.create_vol_geom(vol_dim, vol_dim, vol_dim)

# # Create the projector
# projector_id = astra.create_projector('cuda3d', proj_geom, vol_geom)

# # Example volume data
# volume = np.ones((vol_dim, vol_dim, vol_dim), dtype=np.float32)

# # Create the sinogram
# sinogram_id, sinogram = astra.create_sino3d_gpu(volume, proj_geom, vol_geom)

(512, 10, 512)


In [34]:
sinogram = np.swapaxes(sinogram, 0, 1) # [H, N, W] -> [N, H, W]
print(sinogram.shape)

(10, 512, 512)
